# Homework 10b: Time Series/Classification Modeling

**Assignment**: Build time series model with lag features and classification pipeline.

## Objectives
- Create lag and rolling window features
- Build time series prediction pipeline
- Implement classification model for return direction
- Evaluate temporal model performance

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import utils
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("⏰ Homework 10b: Time Series/Classification Modeling")

## 1. Load and Prepare Time Series Data

In [ ]:
# Load financial data
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
raw_data = utils.fetch_multiple_stocks(symbols, prefer_alphavantage=False, period='2y')

if not raw_data.empty:
    # Process data with time series features
    processed_data = []
    
    for symbol in symbols:
        symbol_data = raw_data[raw_data['symbol'] == symbol].copy().sort_values('date')
        
        # Basic features
        symbol_data['daily_return'] = symbol_data['close'].pct_change()
        symbol_data['log_return'] = np.log(symbol_data['close'] / symbol_data['close'].shift(1))
        symbol_data['price_range'] = (symbol_data['high'] - symbol_data['low']) / symbol_data['close']
        
        # Lag features (previous day values)
        for lag in [1, 2, 3, 5]:
            symbol_data[f'return_lag_{lag}'] = symbol_data['daily_return'].shift(lag)
            symbol_data[f'volume_lag_{lag}'] = symbol_data['volume'].shift(lag)
        
        # Rolling window features
        for window in [5, 10, 20]:
            symbol_data[f'return_mean_{window}'] = symbol_data['daily_return'].rolling(window).mean()
            symbol_data[f'return_std_{window}'] = symbol_data['daily_return'].rolling(window).std()
            symbol_data[f'volume_mean_{window}'] = symbol_data['volume'].rolling(window).mean()
            symbol_data[f'price_sma_{window}'] = symbol_data['close'].rolling(window).mean()
        
        # Technical indicators
        symbol_data['rsi'] = calculate_rsi(symbol_data['close'], window=14)
        symbol_data['macd'] = calculate_macd(symbol_data['close'])
        
        # Target variables
        symbol_data['target_return'] = symbol_data['daily_return'].shift(-1)
        symbol_data['target_direction'] = (symbol_data['target_return'] > 0).astype(int)
        
        processed_data.append(symbol_data)
    
    df = pd.concat(processed_data, ignore_index=True)
    df = df.dropna()
    
    print(f"✅ Time series dataset prepared: {df.shape}")
    print(f"Features: {len([col for col in df.columns if col not in ['symbol', 'date', 'open', 'high', 'low', 'close', 'volume']])}")
else:
    print("❌ Failed to load data")

def calculate_rsi(prices, window=14):
    """Calculate Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD indicator"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    return macd

## 2. Time Series Feature Engineering

In [ ]:
if not df.empty:
    # Define feature sets
    lag_features = [col for col in df.columns if 'lag_' in col]
    rolling_features = [col for col in df.columns if any(x in col for x in ['_mean_', '_std_', '_sma_'])]
    technical_features = ['rsi', 'macd', 'price_range']
    
    all_features = lag_features + rolling_features + technical_features + ['daily_return']
    
    print(f"📊 Feature Categories:")
    print(f"Lag features: {len(lag_features)}")
    print(f"Rolling features: {len(rolling_features)}")
    print(f"Technical features: {len(technical_features)}")
    print(f"Total features: {len(all_features)}")
    
    # Create feature matrix
    X = df[all_features].copy()
    y_regression = df['target_return'].copy()
    y_classification = df['target_direction'].copy()
    
    # Remove NaN values
    mask = ~(X.isna().any(axis=1) | y_regression.isna() | y_classification.isna())
    X = X[mask]
    y_regression = y_regression[mask]
    y_classification = y_classification[mask]
    dates = df[mask]['date']
    symbols = df[mask]['symbol']
    
    print(f"\n📈 Final dataset: {X.shape}")
    print(f"Regression target: {y_regression.shape}")
    print(f"Classification target: {y_classification.shape}")
    print(f"Class distribution: {y_classification.value_counts().to_dict()}")

## 3. Time Series Cross-Validation Split

In [ ]:
if not df.empty:
    # Time series split (preserving temporal order)
    split_date = dates.quantile(0.8)
    train_mask = dates <= split_date
    test_mask = dates > split_date
    
    X_train = X[train_mask]
    X_test = X[test_mask]
    y_reg_train = y_regression[train_mask]
    y_reg_test = y_regression[test_mask]
    y_clf_train = y_classification[train_mask]
    y_clf_test = y_classification[test_mask]
    
    print(f"🔄 Time Series Split:")
    print(f"Train period: {dates[train_mask].min()} to {dates[train_mask].max()}")
    print(f"Test period: {dates[test_mask].min()} to {dates[test_mask].max()}")
    print(f"Train samples: {X_train.shape[0]}")
    print(f"Test samples: {X_test.shape[0]}")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("\n⚖️ Features scaled for modeling")

## 4. Time Series Regression Model

In [ ]:
if not df.empty:
    # Random Forest Regressor for time series
    rf_regressor = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        random_state=42,
        n_jobs=-1
    )
    
    # Fit model
    rf_regressor.fit(X_train_scaled, y_reg_train)
    
    # Predictions
    y_reg_train_pred = rf_regressor.predict(X_train_scaled)
    y_reg_test_pred = rf_regressor.predict(X_test_scaled)
    
    # Regression metrics
    train_r2 = r2_score(y_reg_train, y_reg_train_pred)
    test_r2 = r2_score(y_reg_test, y_reg_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_reg_train, y_reg_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_test_pred))
    
    print("📈 Time Series Regression Results:")
    print(f"Train R²: {train_r2:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.6f}")
    print(f"Test RMSE: {test_rmse:.6f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': all_features,
        'importance': rf_regressor.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔝 Top 10 Most Important Features:")
    print(feature_importance.head(10).round(4))
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Predictions vs Actual
    axes[0].scatter(y_reg_test, y_reg_test_pred, alpha=0.6, s=20)
    axes[0].plot([y_reg_test.min(), y_reg_test.max()], [y_reg_test.min(), y_reg_test.max()], 'r--', lw=2)
    axes[0].set_xlabel('Actual Returns')
    axes[0].set_ylabel('Predicted Returns')
    axes[0].set_title(f'Time Series Regression (R² = {test_r2:.4f})')
    
    # Feature importance
    top_features = feature_importance.head(10)
    axes[1].barh(top_features['feature'], top_features['importance'])
    axes[1].set_xlabel('Feature Importance')
    axes[1].set_title('Top 10 Feature Importances')
    
    plt.tight_layout()
    plt.show()

## 5. Classification Model for Return Direction

In [ ]:
if not df.empty:
    # Random Forest Classifier
    rf_classifier = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    
    # Fit model
    rf_classifier.fit(X_train_scaled, y_clf_train)
    
    # Predictions
    y_clf_train_pred = rf_classifier.predict(X_train_scaled)
    y_clf_test_pred = rf_classifier.predict(X_test_scaled)
    y_clf_test_proba = rf_classifier.predict_proba(X_test_scaled)[:, 1]
    
    # Classification metrics
    train_accuracy = accuracy_score(y_clf_train, y_clf_train_pred)
    test_accuracy = accuracy_score(y_clf_test, y_clf_test_pred)
    
    print("🎯 Classification Results:")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    print("\n📊 Detailed Classification Report:")
    print(classification_report(y_clf_test, y_clf_test_pred, 
                              target_names=['Down', 'Up']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_clf_test, y_clf_test_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Confusion Matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'],
                ax=axes[0])
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].set_title('Confusion Matrix')
    
    # Probability distribution
    axes[1].hist(y_clf_test_proba[y_clf_test == 0], alpha=0.7, label='Down', bins=30)
    axes[1].hist(y_clf_test_proba[y_clf_test == 1], alpha=0.7, label='Up', bins=30)
    axes[1].set_xlabel('Predicted Probability (Up)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Probability Distribution by Class')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Feature importance for classification
    clf_feature_importance = pd.DataFrame({
        'feature': all_features,
        'importance': rf_classifier.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔝 Top 10 Features for Classification:")
    print(clf_feature_importance.head(10).round(4))

## 6. Time Series Cross-Validation

In [ ]:
if not df.empty:
    # Time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    cv_scores_reg = []
    cv_scores_clf = []
    
    print("🔄 Time Series Cross-Validation:")
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_scaled)):
        # Split data
        X_cv_train, X_cv_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_reg_cv_train, y_reg_cv_val = y_reg_train.iloc[train_idx], y_reg_train.iloc[val_idx]
        y_clf_cv_train, y_clf_cv_val = y_clf_train.iloc[train_idx], y_clf_train.iloc[val_idx]
        
        # Regression
        rf_reg_cv = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
        rf_reg_cv.fit(X_cv_train, y_reg_cv_train)
        reg_score = rf_reg_cv.score(X_cv_val, y_reg_cv_val)
        cv_scores_reg.append(reg_score)
        
        # Classification
        rf_clf_cv = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        rf_clf_cv.fit(X_cv_train, y_clf_cv_train)
        clf_score = rf_clf_cv.score(X_cv_val, y_clf_cv_val)
        cv_scores_clf.append(clf_score)
        
        print(f"Fold {fold+1}: Regression R² = {reg_score:.4f}, Classification Acc = {clf_score:.4f}")
    
    print(f"\n📊 Cross-Validation Summary:")
    print(f"Regression R² - Mean: {np.mean(cv_scores_reg):.4f} ± {np.std(cv_scores_reg):.4f}")
    print(f"Classification Acc - Mean: {np.mean(cv_scores_clf):.4f} ± {np.std(cv_scores_clf):.4f}")
    
    # Visualization of CV scores
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].plot(range(1, 6), cv_scores_reg, 'o-', label='CV Scores')
    axes[0].axhline(y=np.mean(cv_scores_reg), color='red', linestyle='--', label='Mean')
    axes[0].set_xlabel('Fold')
    axes[0].set_ylabel('R² Score')
    axes[0].set_title('Regression Cross-Validation')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(range(1, 6), cv_scores_clf, 'o-', label='CV Scores')
    axes[1].axhline(y=np.mean(cv_scores_clf), color='red', linestyle='--', label='Mean')
    axes[1].set_xlabel('Fold')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Classification Cross-Validation')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Model Performance by Symbol

In [ ]:
if not df.empty:
    # Analyze performance by symbol
    test_symbols = symbols[test_mask]
    symbol_performance = []
    
    for symbol in symbols:
        symbol_test_mask = test_symbols == symbol
        if symbol_test_mask.sum() > 10:  # Minimum samples
            # Get symbol-specific predictions
            symbol_reg_actual = y_reg_test[symbol_test_mask]
            symbol_reg_pred = y_reg_test_pred[symbol_test_mask]
            symbol_clf_actual = y_clf_test[symbol_test_mask]
            symbol_clf_pred = y_clf_test_pred[symbol_test_mask]
            
            # Calculate metrics
            reg_r2 = r2_score(symbol_reg_actual, symbol_reg_pred)
            clf_acc = accuracy_score(symbol_clf_actual, symbol_clf_pred)
            
            symbol_performance.append({
                'Symbol': symbol,
                'Regression_R2': reg_r2,
                'Classification_Acc': clf_acc,
                'N_samples': symbol_test_mask.sum()
            })
    
    symbol_df = pd.DataFrame(symbol_performance)
    print("\n📈 Performance by Symbol:")
    print(symbol_df.round(4))
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].bar(symbol_df['Symbol'], symbol_df['Regression_R2'], alpha=0.7)
    axes[0].set_ylabel('R² Score')
    axes[0].set_title('Regression Performance by Symbol')
    axes[0].tick_params(axis='x', rotation=45)
    
    axes[1].bar(symbol_df['Symbol'], symbol_df['Classification_Acc'], alpha=0.7)
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Classification Performance by Symbol')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 8. Summary and Model Comparison

### Time Series Modeling Results

**Regression Model (Return Prediction)**
- Predicts continuous return values using lag and rolling features
- Random Forest captures non-linear relationships
- Feature importance reveals most predictive time series patterns

**Classification Model (Direction Prediction)**
- Predicts binary up/down movement direction
- Balanced classes to handle market symmetry
- Probability outputs enable confidence-based trading strategies

### Key Insights

1. **Temporal Dependencies**: Lag features capture short-term momentum
2. **Rolling Statistics**: Moving averages and volatility provide context
3. **Technical Indicators**: RSI and MACD add market sentiment information
4. **Cross-Validation**: Time series splits preserve temporal structure

### Model Comparison

| Model Type | Strengths | Limitations |
|------------|-----------|-------------|
| Linear Regression | Interpretable, fast | Assumes linearity |
| Time Series RF | Captures non-linearity | Less interpretable |
| Classification | Clear trading signals | Binary output only |

### Practical Applications

1. **Portfolio Management**: Use regression for position sizing
2. **Risk Management**: Classification for stop-loss triggers
3. **Strategy Development**: Combine both models for robust signals
4. **Performance Attribution**: Feature importance guides strategy focus

### Next Steps

1. **Ensemble Methods**: Combine multiple model predictions
2. **Deep Learning**: LSTM/GRU for complex temporal patterns
3. **Alternative Data**: Incorporate sentiment, news, economic indicators
4. **Real-time Implementation**: Deploy models for live trading